In [1]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import re
import openpyxl

### SUBIR PADRON SSS

In [14]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo =  archivo.name # es para obtner la direccion correcta del archivo con extencion.
        columnas = [    "Código de Obra Social",
                        "CUIT del empleador",
                        "CUIL del Titular",
                        "Código de Parentesco",
                        "Cuil",
                        "Tipo Documento",
                        "Número Documento",
                        "Apellido y Nombre",
                        "Sexo",
                        "Estado Civil",
                        "Fecha de Nacimiento",
                        "Nacionalidad",
                        "Calle",
                        "Número de puerta",
                        "Piso",
                        "Departamento",
                        "Localidad",
                        "Código Postal",
                        "Provincia",
                        "Tipo de Domicilio",
                        "Teléfono",
                        "Situación revista",
                        "Incapacidad",
                        "Tipo de beneficiario titular",
                        "Fecha de alta en la Obra Social",
                        "Fecha de cierre de la presentación",
                        "Valida CUIL",
                        "CUIL informado por la OS",
                        "Tipo beneficiario informado en la DDJJ (Declaración Jurada del empleador)",
                        "CUIT empleador informante en la DDJJ",
                        "Obra Social declarada en la DDJJ",
                        "Ultimo período declarado por el empleador en la DDJJ",
                        "Obra Social elegida (en caso de opción)",
                        "Período desde de vigencia de la opción"
                        ]
        try:
            padron_sss = pd.read_csv(ruta_archivo, sep='|', names=columnas)

            print (padron_sss)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/Downloads/127406-202308-PC.txt' mode='r' encoding='cp1252'>
       Código de Obra Social  CUIT del empleador  CUIL del Titular  \
0                     127406         20120810805       27264313061   
1                     127406         30714444162       20214552478   
2                     127406         30714444162       20214552478   
3                     127406         30714444162       20214552478   
4                     127406         30570955388       20246881422   
...                      ...                 ...               ...   
15374                 127406         30535218044       27256232761   
15375                 127406         30516283781       20275000303   
15376                 127406         27922688082       27942342174   
15377                 127406         30521422390       20339323055   
15378                 127406         30712103317       27356670723   

       Código de Parentesco         Cuil Tipo Documento  Nú

## cargar padron de sidefi

In [15]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron = pd.read_excel(ruta_archivo)
            padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/Downloads/sidefi_padron_altas_noviembre.csv' mode='r' encoding='cp1252'>
       Nro. OS          Plan del Afiliado Opcion  Nro. Grupo Familiar  \
0            7  Monotributista social EPV     SI             26407599   
1            7  Monotributista social EPV     SI             26407599   
2            7  Monotributista social EPV     SI             26407599   
3           10  Monotributista social MEP     SI             31403090   
4            1      Monotributista social     NO             36821087   
...        ...                        ...    ...                  ...   
16255        1    Monotributista autonomo     NO             92744412   
16256        1                   Afiliado     NO             46122207   
16257        1                   Afiliado     NO             42629799   
16258        1                   Afiliado     SI             36602782   
16259        7  Monotributista social EPV     SI             32749818   

         N

###  transformar dependencia en sidefi

In [16]:
padron['Dependencia']=padron['Dependencia'].astype(str)

dependencias = {
"10" : "CABA",
"20" : "ZONA NORTE Y NOROESTE",
"30" : "ZONA OESTE",
"40" : "ZONA SUR",
"50" : "LA PLATA",
"60" : "MAR DEL PLATA",
"70" : "PERGAMINO",
"80" : "ROSARIO",
"90" : "CORDOBA",
"100" : "MENDOZA",
"110" : "TUCUMAN"
}

padron['Dependencia']=padron['Dependencia'].replace(dependencias)
padron['Dependencia'].head(5)



0    ZONA NORTE Y NOROESTE
1    ZONA NORTE Y NOROESTE
2    ZONA NORTE Y NOROESTE
3                 ZONA SUR
4               ZONA OESTE
Name: Dependencia, dtype: object

## Cruzar los datos 

In [17]:
cruce_sidefi = pd.merge(padron, padron_sss, left_on='Nro. Documento', right_on='Número Documento', how='outer', indicator=True)
cruce_sidefi
save_path = filedialog.askdirectory()
cruce_sidefi.to_excel(save_path + '\cruce_sss.xlsx', engine='openpyxl' ,index = False )
cruce_sidefi.dtypes

Nro. OS                                                  float64
Plan del Afiliado                                         object
Opcion                                                    object
Nro. Grupo Familiar                                      float64
Nro. Afiliado                                             object
                                                          ...   
Obra Social declarada en la DDJJ                         float64
Ultimo período declarado por el empleador en la DDJJ     float64
Obra Social elegida (en caso de opción)                  float64
Período desde de vigencia de la opción                   float64
_merge                                                  category
Length: 69, dtype: object

In [18]:
cruce_sidefi['Tipo de beneficiario titular'].fillna(20, inplace=True)
cruce_sidefi['Tipo de beneficiario titular'].astype(int)


0         7
1         7
2        20
3        20
4         7
         ..
18847     7
18848     0
18849     0
18850     0
18851     0
Name: Tipo de beneficiario titular, Length: 18852, dtype: int32

In [20]:
cruce_sidefi['Dependencia'].head(5)
tipo_benef= {
0:"RELACION DE DEPENDENCIA",
1:"PASANTES",
2:"JUBILADOS DEL SISTEMA NACIONAL DEL SEGURO DE SALUD",
3:"ADHERENTES VOLUNTARIOS",
4:"PEQUEÑOS CONTRIBUYENTES – MONOTRIBUTISTAS (LEY 25.865)",
5:"SERVICIO DOMESTICO (LEY 26.068)",
6:"JUBILADOS DEL REGISTRO PARA LA ATENCION DE ESPAÑOLES",
7:"MONOTRIBUTISTAS EFECTORES SOCIALES",
8:"BENEFICIARIOS DE SEGURO DE DESEMPLEO (LEY 24.013)",
9:"BENEFICIARIOS DE SEGURO DE DESEMPLEO (LEY 25.191) (RENATRE)",
10:"JUBILADOS Y PENSIONADOS DEL INSTITUTO NACIONAL DE SERVICIOS SOCIALES",
11:"JUBILADOS Y PENSIONADOS DE OTROS SISTEMAS",
12:"RELACION DE DEPENDENCIA (SIN APORTES Y CONTRIBUCION POR SJP)",
20:""
}
cruce = {
    "left_only":"Sidefi",
    "both":"ambos",
    "right_only":"SSS"
    }


cruce_sidefi['Tipo de beneficiario titular'].replace(tipo_benef, inplace=True)
cruce_sidefi=cruce_sidefi[["Nro. OS",
"Dependencia",
"Plan del Afiliado",
"Opcion",
"Apellido Afiliado",
"Nombre Afiliado",
"Nro. Documento",
"Fecha Nacimiento",
"Fecha Alta",
"Fecha Baja",
"Número Documento",
"Apellido y Nombre",
"Incapacidad",
"Tipo de beneficiario titular",
"Fecha de alta en la Obra Social",
"_merge"
]]


In [21]:
groups = cruce_sidefi.groupby('Dependencia') # Agrupar el dataframe por la columna convenio
dfs = [] # Crear una lista vacía para almacenar los dataframes separados
for name, group in groups: # Iterar sobre los grupos y añadir cada dataframe a la lista
    dfs.append(group)
    print(dfs) # Imprimir la lista de dataframes

save_path = filedialog.askdirectory()
cruce_sidefi.to_excel(save_path + '\devoluciones2_septiembre2023.xlsx', engine='openpyxl' ,index = False )

### guardarlos con los nombres correspondientes
for df in dfs: # Iterar sobre la lista de dataframes
    sede = df['Dependencia'].iloc[0] # Obtener el nombre del convenio del primer elemento del grupo
    filename = save_path + '/' + sede + '.xlsx' # Crear el nombre del archivo con la ruta de la carpeta y el nombre del convenio
    df.to_excel(filename, index=False) # Guardar el dataframe en el archivo excel sin el índice
    print(f"Se ha guardado el archivo {filename}") # Imprimir un mensaje de confirmación

[       Nro. OS Dependencia          Plan del Afiliado Opcion  \
5         10.0        CABA  Monotributista social MEP     SI   
6          1.0        CABA                   Afiliado     NO   
7          1.0        CABA                   Afiliado     NO   
13         1.0        CABA                   Afiliado     NO   
18         1.0        CABA                   Afiliado     NO   
...        ...         ...                        ...    ...   
16304      1.0        CABA    Monotributista autonomo     NO   
16305      1.0        CABA    Monotributista autonomo     NO   
16306      1.0        CABA                   Afiliado     NO   
16308      1.0        CABA                   Afiliado     SI   
16309      7.0        CABA  Monotributista social EPV     SI   

      Apellido Afiliado  Nombre Afiliado  Nro. Documento Fecha Nacimiento  \
5                ABALOS  CLAUDIO ALBERTO      31015657.0        23/7/1984   
6                 ABATE     GISELA NOELI      35995281.0         2/2/1988   